# 配置环境配置

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# proj_dir = "/content/drive/MyDrive/Colab Notebooks"

proj_dir = "/home/ubuntu/git/DBMA"

# 安装包和配置OpenAI

In [2]:
# %pip uninstall agency-swarm -y
# %pip install git+https://github.com/dcstrange/agency-swarm.git@dbOps_Agency #AgencySwarm
# %pip install -q gradio # agency 会话前端
# #%pip install -q duckduckgo_search # Web searching

import os
proj_dir = "/home/ubuntu/git/DBMA"
secret_dir = proj_dir + "/secret-configs"
with open(secret_dir + "/OAI_API_KEY") as f:
    os.environ['OPENAI_API_KEY'] = f.read()

from agency_swarm import set_openai_key
set_openai_key(os.environ['OPENAI_API_KEY'])

# Or
# from getpass import getpass
# set_openai_key(getpass("Please enter your openai key: "))

# 🔨Pseudo-database Environment
💡This is a pseudo-database environment implemented by manual input and output just for a Demo version.
This interface delegates specific tasks requested by other Agents to be performed manually.
This API will be replaced by a real Agent in Dev version, which is responsible for interacting with the database environment.

这是一个通过手动输入和输出实现的伪数据库环境，仅用于Demo版本。此接口委托其他代理要求手动执行的特定任务。这个 API 在Dev版本中会被一个真正的代理所替代，代理负责与数据库环境进行交互。

In [3]:
from agency_swarm import BaseTool
from pydantic import Field
from db_pseudo_env.db_task_api import Assign_DB_Task

class InteractWithDataBaseEnv(BaseTool):
    """Use this tool to interact with database environment.
    This tool recieve the text of the specific action (eg, command, scripts or third-party tools), it then execute in the database environment, and it will finally return the result."""

    specific_task: str = Field(..., description="The specific task interacting with database environment.")


    # This code will be executed if the agent calls this tool
    def run(self):
      return str(Assign_DB_Task(self.specific_task)['response'])


In [4]:
agent_db_env_instructions = """ #Instructions for Database Environment Agent

### Task: Call the function or tools to interact with database environment.
### IMPORTANT: If a tool is used, return the results of the tool's execution to the CEO agent as is, without any analysis or processing.

You are a highly specialized GPT focused on SQL database operations for professional database administrators and developers.
You understand specific intents recieved from other agents and generates appropriate database actions using commands, scripts, or third-party tools.
Your primary objective is to interact effectively with SQL databases, providing accurate and advanced technical solutions.
you focuses on returning precise results and analyzing them for technical users.
You maintains a professional and straightforward tone, prioritizing technical accuracy and practicality over personalization, ensuring that its responses are detailed, relevant, and valuable to database professionals.
"""


In [5]:
from agency_swarm import Agent

agent_db_env = Agent(name="Database Environment",
                     tools=[InteractWithDataBaseEnv],
                     description="responsible for taking specific tasks and executing them in the database environment (including databases, monitoring systems, etc.), planning and executing specific actions.",
                     instructions=agent_db_env_instructions,
                     files_folder=None)


# CEO Agent


In [6]:
ceo_instructions = """# Instructions for CEO Agent

- Ensure that proposal is send to the user before proceeding with task execution.
- Delegate tasks to appropriate agents, ensuring they align with their expertise and capabilities.
- Clearly define the objectives and expected outcomes for each task.
- Provide necessary context and background information for successful task completion.
- Maintain ongoing communication with agents until complete task execution.
- Review completed tasks to ensure they meet the set objectives.
- Report the results back to the user."""

In [7]:
from agency_swarm import Agent

ceo = Agent(name="CEO",
            description="Responsible for client communication, task planning and management.",
            instructions=ceo_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[])

# CoT Task Agent

In [8]:
CoT_instructions = """# Instruction for LLM to Create an Agent for MySQL O&M Task Planning

1. **Objective**: You're an Agent specialized in MySQL Operations & Maintenance (O&M) task planning.
2. **Input Processing**:
    - Receive and interpret O&M task intents from users.
    - Analyze the task requirements and context.
3. **Task Planning Using Chain of Thought (CoT)**:
    - Utilize the Chain of Thought technique for planning.
    - Draw on expert knowledge and previous cases to form a logical, step-by-step task chain.
    - Ensure each step in the task chain is clear, actionable, and relevant to the overall O&M task.
4. **Interaction with Database Environment**:
    - Dispatch specific tasks from the chain to the Agent operating within the database environment.
    - Each task should be formulated to interact effectively with the database, considering current state and potential issues.
5. **Dynamic Adjustment Based on Feedback**:
    - Continuously monitor the feedback from the database environment.
    - Adjust the subsequent tasks in the chain based on the results and feedback received.
    - Ensure the adjustment process is agile and responsive to real-time changes in the database environment.
6. **Success Criteria**:
    - The task chain leads to the successful execution of the entire O&M task.
    - Efficiency and accuracy in task execution.
    - Minimal disruptions and optimal performance of the MySQL database during and after the O&M activities.
7. **Continual Learning and Improvement**:
    - Implement mechanisms for the Agent to learn from each completed task.
    - Update the knowledge base and CoT strategies based on new experiences and outcomes.
"""


agent_CoT = Agent(name="CoT Task Agent",
                     tools=[],
                     description="responsible for MySQL Operations & Maintenance (O&M) task planning by CoT, and send the specific task to the database environment. ",
                     instructions=CoT_instructions,
                     files_folder=None)


# Create Agency

In [9]:
agency_manifesto = """You are tasked with solving O&M problems for MySQL users."""

In [10]:
from agency_swarm import Agency

agency = Agency([
    ceo,
    [ceo, agent_CoT],
    [agent_CoT, agent_db_env]
], shared_instructions=agency_manifesto)

In [11]:
agency.demo_gradio(height=900)

/home/ubuntu/git/DBMA/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Text sent successfully, check the http://127.0.0.1:5000/task.
Reply: ### Simulated SQL Command Results

#### SHOW GLOBAL VARIABLES LIKE 'slow_query_log';

| Variable_name   | Value |
|-----------------|-------|
| slow_query_log  | ON    |

#### SHOW GLOBAL VARIABLES LIKE 'long_query_time';

| Variable_name   | Value |
|-----------------|-------|
| long_query_time | 2.0   |

#### SHOW GLOBAL VARIABLES LIKE 'slow_query_log_file';

| Variable_name       | Value                               |
|---------------------|-------------------------------------|
| slow_query_log_file | /var/log/mysql/mysql-slow.log       |

#### SELECT start_time, user_host, query_time, lock_time, rows_sent, rows_examined, db, sql_text FROM mysql.slow_log WHERE db = 'MegaShopDB' AND (sql_text LIKE '%product_listings%' OR sql_text LIKE '%customer_orders%') ORDER BY start_time DESC;

| start_time           | user_host       | query_time | lock_time | rows_sent | rows_examined | db         | sql_text                 